<br>

<div align=center><font color=maroon size=6><b>AD in Pytorch</b></font></div>

<font size=3>**Author:** Breandan Considine

Referenced from <a href="https://compcalc.github.io/" style="text-decoration:none;">Mila Computer Calculus RG</a></font> &emsp;[[Original notebook link]](https://colab.research.google.com/drive/14KPZmXxa21YkFGKsVe37RV-AiT6YMRao#scrollTo=fKa_PFVfKhJU)

<br>

# Picograd

In [1]:
from functools import reduce

In [2]:
class Var:
    
    def __init__(self, val, grad_fn=lambda: []): # grad_fn 返回值是对应函数各变量的偏导数构成的向量
        self.v, self.grad_fn = val, grad_fn

    def __add__(self, other):
        # 两变量相加，对各自的偏导数都是 1。
        return Var(self.v + other.v, lambda: [(self, 1.0), (other, 1.0)])

    def __mul__(self, other):
        # 两变量相乘，对各自的偏导数结果则是另一个变量
        # 如：f = x*y
        # ∂f/∂x = y
        # ∂f/∂y = x
        return Var(self.v * other.v, lambda: [(self, other.v), (other, self.v)])

    def grad(self, bp=1.0):
        grads = [input.grad(bp=val * bp) for (input, val) in self.grad_fn()]
        merge = lambda a, b: {k: a.get(k, 0) + b.get(k, 0) for k in set(a) | set(b)}
        return reduce(merge, grads, {self: bp})

In [3]:
x = Var(1.)
y = Var(1.)
f = x * x * x * y * x + y * y * y
grad = f.grad()

print(grad[x])
print(grad[y])
print(f.v)

4.0
4.0
2.0


In [4]:
f.grad_fn

<function __main__.Var.__add__.<locals>.<lambda>()>

In [5]:
f.grad_fn()

[(<__main__.Var at 0x17cde32e580>, 1.0),
 (<__main__.Var at 0x17cde32e430>, 1.0)]

<br>

## Exercise #0

Implement linear regression using Picograd and compare with PyTorch.

In [6]:
import torch.nn as nn

torchlr = nn.Linear(3, 3)
inputs = [1., 1., 1.]

def picolr(x):
  # Your code goes here
  return []

# assert torch.allclose(picolr(inputs), torchlr(torch.tensor([inputs])))

<br>
<br>
<br>

# PyTorch

In [7]:
from typing import Callable
import torch
# Certain features in this tutorial are experimental. We will supress the warning.
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

from torch._vmap_internals import vmap

<br>
<br>
<br>

# Vectorizing Maps (vmap)

<font size=3>A `vmap` takes a function and returns a function which accepts a tensor, and maps that function over the tensor (elementwise by default).</font>

In [8]:
ten = torch.ones(3, 3)
fun = lambda x: x + 3
vmap(fun)(ten)

tensor([[4., 4., 4.],
        [4., 4., 4.],
        [4., 4., 4.]])

<br>

<font size=3>`vmap` is composable with most things, including itself.</font>

In [9]:
vmap(fun)(vmap(fun)(ten))

tensor([[7., 7., 7.],
        [7., 7., 7.],
        [7., 7., 7.]])

In [10]:
vmap(vmap(fun))(ten)

tensor([[4., 4., 4.],
        [4., 4., 4.],
        [4., 4., 4.]])

<br>

<font size=3>`vmap` also works on polyadic lambdas:</font>

In [11]:
x = torch.randn(3, 3)
y = torch.randn(3, 3)

print(x)

print(y)

tensor([[ 0.7202,  0.1786, -0.7338],
        [ 0.1254,  0.7197, -0.1640],
        [ 0.2520,  0.1262, -1.7422]])
tensor([[ 0.9401, -0.5548,  0.0454],
        [ 0.7442,  0.8675, -0.5645],
        [-0.3177,  1.2194,  0.0376]])


In [12]:
vmap(torch.mul)(x, y)

tensor([[ 0.6770, -0.0991, -0.0333],
        [ 0.0933,  0.6243,  0.0926],
        [-0.0801,  0.1539, -0.0655]])

In [13]:
x * y

tensor([[ 0.6770, -0.0991, -0.0333],
        [ 0.0933,  0.6243,  0.0926],
        [-0.0801,  0.1539, -0.0655]])

<br>

In [14]:
x = torch.ones(3, 3) * 2
y = torch.ones(3, 3) * 3
z = torch.ones(3, 3) * 4

vmap(lambda x, y, z: x + y + z)(x, y, z)

tensor([[9., 9., 9.],
        [9., 9., 9.],
        [9., 9., 9.]])

<br>

<font size=3>`vmap` can also handle tensor contractions:</font>

In [15]:
torch.dot              # [  D  ],[  D  ] ->   S
vd =  vmap(torch.dot)  # [ N,D ],[ N,D ] -> [ N ]
vvd = vmap(vd)         # [N,D,C],[N,D,C] -> [N,D]
x, y = torch.ones(3, 2, 5), torch.ones(3, 2, 5)
vvd(x, y)

tensor([[5., 5.],
        [5., 5.],
        [5., 5.]])

```python
vd =  vmap(torch.dot)  # [ N,D ],[ N,D ] -> [ N ]
# vd 的参数是一个由2维 torch.Tensor 构成的数组：([ N,D ], [ N,D ])，
# 任何一个2维的 [ N,D ] 都可以看作元素为 [D] 的1维 行 torch.Tensor 堆叠而成的2维 torch.Tensor
# 故 vd =  vmap(torch.dot) 相当于对于每一个参数的元素（1维 行 torch.Tensor 的 [D]）
# 进行 torch.dot 运算，即 [  D  ],[  D  ] ->   S，结果是一个标量。
# 因为任何一个2维的 [ N,D ] 都有 N 行的 [ D ]，故这N个标量又构成一个1维的 torch.Tensor [ N ]


```

<br>

<font size=3>Why should you use `vmap`?</font>

In [16]:
import time

fun = lambda x: x**2 / x *x **x
ten = torch.randn(100000)



start = time.perf_counter()

for i in range(0, 100000):
  ten[i] = fun(ten[i])

print(f"loop completed in {time.perf_counter() - start:0.4f} seconds")



start = time.perf_counter()

vmap(fun)(ten)

print(f"vmap completed in {time.perf_counter() - start:0.4f} seconds")

loop completed in 1.5586 seconds
vmap completed in 0.0010 seconds


<br>

## Exercise #1

Implement matrix multiplication using `vmap` and use this to implement the forward pass of an MLP. Compare numerical results with the object-oriented version.

In [17]:
seq_mlp = nn.Sequential(
        nn.Linear(3, 3),
        nn.Sigmoid(),
        nn.Linear(3, 2),
        nn.Sigmoid()
      )

inputs = torch.ones(3)

# Your code goes here.

# assert torch.allclose(seq_mlp(inputs), vmap_mlp(inputs))

<br>
<br>
<br>

# Gradients

<font size=3>PyTorch generally treats functions as stateful objects. This is the object-oriented way.</font>

In [18]:
def grad_fun(f, a, b, grad):
    
    f.backward()
    return a.grad, b.grad

x = torch.tensor([1., 3.], requires_grad=True)
y = torch.tensor([6., 4.], requires_grad=True)
x.grad, y.grad

(None, None)

<br>

<font size=3>Calling grad_fun mutates the variables...</font>

In [19]:
grad = torch.tensor([1., 1.])
f = sum(x * y)

grad_fun(f, x, y, grad)

x.grad, y.grad

(tensor([6., 4.]), tensor([1., 3.]))

In [20]:
print(f)
print(x)
print(y)

tensor(18., grad_fn=<AddBackward0>)
tensor([1., 3.], requires_grad=True)
tensor([6., 4.], requires_grad=True)


<br>

<font size=3>What happens if we call the function a second time?</font>

In [21]:
grad_fun(f, x, y, grad)

RuntimeError: Trying to backward through the graph a second time (or directly access saved tensors after they have already been freed). Saved intermediate values of the graph are freed when you call .backward() or autograd.grad(). Specify retain_graph=True if you need to backward through the graph a second time or if you need to access saved tensors after calling backward.

<br>

<font size=3>Gradients in PyTorch are lowered onto `torch.autograd.grad`. This is useful for defining custom gradients.</font>

In [22]:
x = torch.randn(2, requires_grad=True)
t = torch.tensor([2., 3.], requires_grad=True)
y = torch.randn(2, requires_grad=True)


f = sum((x*t - y)**2)**0.5
# ∂f/∂x = 1/2 * sum( (x*t-y)**2)**(-1/2) *2 *(x*t-y) *t
# ∂f/∂t = 1/2 * sum( (x*t-y)**2)**(-1/2) *2 *(x*t-y) *x


torch.autograd.grad(f, inputs=[x, t])

(tensor([-1.3858, -2.1631]), tensor([0.4892, 0.3294]))

In [23]:
# ∂f/∂x = 1/2 * sum( (x*t-y)**2)**(-1/2) *2 *(x*t-y) *t
1/2 * sum( (x*t-y)**2)**(-1/2) *2 *(x*t-y) *t

tensor([-1.3858, -2.1631], grad_fn=<MulBackward0>)

In [24]:
# ∂f/∂t = 1/2 * sum( (x*t-y)**2)**(-1/2) *2 *(x*t-y) *x
1/2 * sum( (x*t-y)**2)**(-1/2) *2 *(x*t-y) *x

tensor([0.4892, 0.3294], grad_fn=<MulBackward0>)

In [25]:
torch.autograd.grad(f, inputs=[x, t])

RuntimeError: Trying to backward through the graph a second time (or directly access saved tensors after they have already been freed). Saved intermediate values of the graph are freed when you call .backward() or autograd.grad(). Specify retain_graph=True if you need to backward through the graph a second time or if you need to access saved tensors after calling backward.

<br>
<br>
<br>

# Jacobians

<font size=3>The *Jacobian* $\mathcal{J}: (\mathbb{R}^m\rightarrow\mathbb{R}^n)\rightarrow\mathbb{R}^{n \times m}$ is a function which takes a vector-valued function and returns a matrix of partials, whose rows are gradients of each of the outputs with respect to each of the inputs.</font>

<br>
<br>

<font size=4>
$$
\mathcal{J}\circ\mathbf{f} =
            \begin{bmatrix}
                \dfrac{\partial \mathbf{f}}{\partial x_1} & \cdots & \dfrac{\partial \mathbf{f}}{\partial x_m}
            \end{bmatrix}^\intercal =
            \begin{bmatrix}
                \dfrac{\partial f_1}{\partial x_1} & \cdots & \dfrac{\partial f_1}{\partial x_m}\\
                \vdots & \ddots & \vdots\\
                \dfrac{\partial f_n}{\partial x_1} & \cdots & \dfrac{\partial f_n}{\partial x_m}
            \end{bmatrix} =
            \begin{bmatrix}
                \nabla f_1 \\
                \vdots \\
                \nabla f_m
            \end{bmatrix}
$$</font>

In [26]:
def jacobian(fun, x) -> torch.Tensor:
    
    x = x.detach().requires_grad_()
    y = fun(x)
    vjp = lambda v: torch.autograd.grad(y, x, v)[0]

    vs = torch.eye(y.numel())\
            .view(y.numel(), *y.shape)
    
    print(y.numel())
    print(torch.eye(y.numel()))
    print(vs)
    
    result = vmap(vjp)(vs)
    return result.detach()

<font size=3>Adapted from: https://gist.github.com/zou3519/b8c0c534c7509e7a3ed0612d01226fb7</font>

<br>

In [27]:
f = lambda x: x ** 3
jacobian(f, torch.ones(3))

3
tensor([[1., 0., 0.],
        [0., 1., 0.],
        [0., 0., 1.]])
tensor([[1., 0., 0.],
        [0., 1., 0.],
        [0., 0., 1.]])


tensor([[3., 0., 0.],
        [0., 3., 0.],
        [0., 0., 3.]])

In [28]:
x = torch.randn(3, requires_grad=True)
y = torch.randn(3)

vjp_mul = lambda v: torch.autograd.grad([x * y], [x], grad_outputs=[v])[0]

batched_v = torch.eye(3)
vmap(vjp_mul)(batched_v)

tensor([[-0.9279,  0.0000, -0.0000],
        [-0.0000,  0.6971, -0.0000],
        [-0.0000,  0.0000, -1.1016]])

<br>

## Exercise #2

In [ ]:
# Your code goes here.

# assert torch.allclose(forward_mode(f, x), reverse_mode(f, x))

<br>
<br>
<br>

# Hessians

<font size=3>The *Hessian* $\mathbf{H}:(\mathbb{R}^m\rightarrow\mathbb{R})\rightarrow\mathbb{R}^{m\times m}$ is a higher-order function which maps scalar functions to a matrix of second-order partial derivatives:</font>

<br>
<br>

<font size=4>
$$
\mathbf{H}\circ Q = \begin{bmatrix}{\dfrac {\partial ^{2}Q}{\partial x_{1}^{2}}}
                                &{\dfrac {\partial ^{2}Q}{\partial x_{1}\,\partial x_{2}}}&\cdots &{\dfrac {\partial ^{2}Q}{\partial x_{1}\,\partial x_{m}}}\\[2.2ex]{\dfrac {\partial ^{2}Q}{\partial x_{2}\,\partial x_{1}}}&{\dfrac {\partial ^{2}Q}{\partial x_{2}^{2}}}&\cdots &{\dfrac {\partial ^{2}Q}{\partial x_{2}\,\partial x_{m}}}\\[2.2ex]\vdots &\vdots &\ddots &\vdots \\[2.2ex]{\dfrac {\partial ^{2}Q}{\partial x_{m}\,\partial x_{1}}}&{\dfrac {\partial ^{2}Q}{\partial x_{m}\,\partial x_{2}}}&\cdots &{\dfrac {\partial ^{2}Q}{\partial x_{m}^{2}}}
            \end{bmatrix}
$$</font>


<br>
<br>

Under [certain conditions](https://en.wikipedia.org/wiki/Symmetry_of_second_derivatives#Theorem_of_Schwarz) the Hessian is symmetric, although these conditions do [not always hold](https://math.stackexchange.com/questions/29536/asymmetric-hessian-matrix).

锐平添加：<font size=4>$H(Q)^T=(H∘Q)^T=\mathcal{J}∘∇Q$</font>

<br>

In [29]:
def hessian(fun: Callable, x: torch.Tensor) ->  torch.Tensor:
    def first_grad(x: torch.Tensor):
        y = fun(x)
        assert y.dim() == 0  # hessian only defined on scalar-valued function
        return torch.autograd.grad(y, x, create_graph=True)[0]

    return jacobian(first_grad, x)

<font size=3>Adapted from: https://gist.github.com/zou3519/b8c0c534c7509e7a3ed0612d01226fb7</font>

<br>

In [30]:
g = lambda x: (x ** 3).sum()
hessian(g, torch.ones(3))

3
tensor([[1., 0., 0.],
        [0., 1., 0.],
        [0., 0., 1.]])
tensor([[1., 0., 0.],
        [0., 1., 0.],
        [0., 0., 1.]])


tensor([[6., 0., 0.],
        [0., 6., 0.],
        [0., 0., 6.]])

<br>

## Exercise #3

<br>

<font size=3>Show that $(\mathbf{H} \circ Q)^\intercal = \mathcal{J} \circ \nabla Q$.</font>

In [ ]:
# Your code goes here.

# assert torch.allclose(hessian(g, x), jacobian_gradient(g, x))

<br>
<br>
<br>

# Named Tensors

<font size=3>In `numpy`, it is possible to broadcast two dimensions if their [shapes are equal](https://numpy.org/doc/stable/user/basics.broadcasting.html#general-broadcasting-rules). However keeping track of dimension order and origin can be confusing, i.e. two dimensions may have the same shape but different [semantics](https://en.wikipedia.org/wiki/Covariance_and_contravariance_of_vectors#Use_in_tensor_analysis). [Named tensors](https://namedtensor.github.io/) provide a more user-friendly notation for manipulating tensors.</font>

In [31]:
state = torch.ones(9, 5, names=('batch', 'D'))
trans = torch.randn(5, 5, names=('in', 'out'))
next_state = state @ trans
print(next_state.names)

('batch', 'out')


<br>

<font size=3>In PyTorch, attempting to accumulate named tensors along dimensions with different names will produce a runtime error:</font>

In [32]:
x = torch.ones(3, names=('X',))
y = torch.ones(3, names=('Z',))
z = x + y

RuntimeError: Error when attempting to broadcast dims ['X'] and dims ['Z']: dim 'X' and dim 'Z' are at the same position from the right but do not match.

<br>

<font size=3>General matrix multiplication is a dot product between tensors.</font>

In [33]:
x = torch.randn(3, 3, 3, 3, names=('A', 'B', 'C', 'D'))
y = torch.randn(3, 3, 3, names=('C', 'F', 'G'))
torch.matmul(x, y)

RuntimeError: matmul: Expected 'B' (index 1 of ['A', 'B', 'C', 'D']) to match 'C' (index 0 of ['C', 'F', 'G']) but they do not match.

<br>

## Exercise #4

Implement [2D depthwise convolution](https://www.tensorflow.org/api_docs/python/tf/nn/depthwise_conv2d) using named tensors in PyTorch.

In [34]:
# Your code goes here.

<br>
<br>
<br>